# Classifing last names with character-level RNN

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import pandas as pd
import numpy as np
from pathlib import Path
import torch
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import random

## Dataset
`wget https://github.com/hunkim/PyTorchZeroToAll/blob/master/data/names_train.csv.gz`

`wget https://github.com/hunkim/PyTorchZeroToAll/blob/master/data/names_test.csv.gz`

In [2]:
PATH = Path("/data2/yinterian/name_dataset/")
list(PATH.iterdir())

[PosixPath('/data2/yinterian/name_dataset/names_test.csv'),
 PosixPath('/data2/yinterian/name_dataset/names_train.csv.gz'),
 PosixPath('/data2/yinterian/name_dataset/names_train.csv'),
 PosixPath('/data2/yinterian/name_dataset/names_test.csv.gz')]

In [3]:
! head /data2/yinterian/name_dataset/names_train.csv

### Processing data

In [4]:
df = pd.read_csv(PATH/"names_train.csv", header=None)

In [5]:
# getting a vocabulary of characters
letters = [list(l) for l in df[0].values]
vocab = sorted(list(set(np.concatenate(np.array(letters)))))
vocab[:10]

[' ', "'", ',', 'A', 'B', 'C', 'D', 'E', 'F', 'G']

In [6]:
vocab2id = {key:i for i, key in enumerate(vocab)}
vocab2id[" "] # I am going to use 0 to pad sequences

0

In [7]:
labels = sorted(df[1].unique())
label2id = {key:i for i, key in enumerate(labels)}
label2id

{'Arabic': 0,
 'Chinese': 1,
 'Czech': 2,
 'Dutch': 3,
 'English': 4,
 'French': 5,
 'German': 6,
 'Greek': 7,
 'Irish': 8,
 'Italian': 9,
 'Japanese': 10,
 'Korean': 11,
 'Polish': 12,
 'Portuguese': 13,
 'Russian': 14,
 'Scottish': 15,
 'Spanish': 16,
 'Vietnamese': 17}

In [8]:
def pad_seq(x, seq_len=15, vocab2id=vocab2id):
    x = list(x)
    x = np.array([vocab2id[k] for k in x])
    z = np.zeros(seq_len, dtype=np.int32)
    n = min(seq_len, x.shape[0])
    z[seq_len - n:] = x[0:n]
    return z

In [9]:
x = pad_seq("aabbb")
x

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 29, 29, 30, 30, 30],
      dtype=int32)

In [10]:
# one hot encoding
def seq2matrix(x, vocab_len=55):
    z = np.zeros((x.shape[0], vocab_len))
    z[np.arange(len(x)), x] = 1
    return z

In [11]:
class NameDataset(Dataset):
    def __init__(self, path, vocab2id, label2id, seq_len=15, vocab_len=55):
        self.df = pd.read_csv(path, header=None)
        self.label2id = label2id
        self.vocab2id = vocab2id
        self.seq_len = seq_len
        self.vocab_len = vocab_len 
        self.x = df[0].values
        self.y = [self.label2id[l] for l in df[1].values]
        self.vocab2id = vocab2id
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        x = pad_seq(self.x[idx], self.seq_len, self.vocab2id)
        x = seq2matrix(x, self.vocab_len)
        return x, self.y[idx]

In [12]:
train = NameDataset(PATH/"names_train.csv", vocab2id, label2id)
test = NameDataset(PATH/"names_test.csv", vocab2id, label2id)

In [13]:
batch_size = 2000
n = len(test)
train_dl = DataLoader(train, batch_size=batch_size)
test_dl = DataLoader(test, batch_size=n)

In [14]:
len(train), len(test)

(13374, 13374)

In [15]:
x,y = train[0]
print(x.shape,y)

(15, 55) 2


In [16]:
x

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0

## Model

In [24]:
class CharRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(CharRNN, self).__init__()

        self.hidden_size = hidden_size
        self.linear_i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.linear_h2o = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden):
        combined = torch.cat((x, hidden), 1)
        hidden = torch.tanh(self.linear_i2h(combined))
        output = self.linear_h2o(hidden)
        return output, hidden

    def initHidden(self, bash_size):
        return torch.zeros(bash_size, self.hidden_size)

## Debugging model

In [25]:
vocab_size = 55
hidden_size = 100
n_classes = 18
model = CharRNN(vocab_size, hidden_size, n_classes).cuda()

In [26]:
x, y = next(iter(train_dl))

In [27]:
x.shape, y.shape

(torch.Size([2000, 15, 55]), torch.Size([2000]))

In [28]:
batch = x.shape[0]
h = model.initHidden(batch).cuda()
x = x.cuda().float()
y = y.cuda().long()

In [29]:
torch.cat((x[:,0], h), 1).size()

torch.Size([2000, 155])

In [30]:
for ei in range(x.shape[1]):
    x_t, h = model(x[:,ei], h)

In [31]:
loss = F.cross_entropy(x_t, y)
loss

tensor(2.9295, device='cuda:0', grad_fn=<NllLossBackward>)

## Training

In [32]:
vocab_size = 55
hidden_size = 100
n_classes = 18
model = CharRNN(vocab_size, hidden_size, n_classes).cuda()

In [33]:
def get_optimizer(model, lr = 0.01, wd = 0.00001):
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optim = torch.optim.Adam(parameters, lr=lr, weight_decay=wd)
    return optim

In [34]:
def train(model, optim, train_dl):
    model.train()
    total = 0
    sum_loss = 0
    for x, y in train_dl:
        batch = x.shape[0]
        h = model.initHidden(batch).cuda()
        loss = 0
        x = x.cuda().float()
        y = y.cuda().long()
        
        for t in range(x.shape[1]):
            out, h = model(x[:,t], h)
        
        loss = F.cross_entropy(out, y)
        optim.zero_grad()
        loss.backward()
        optim.step()
        total += batch
        sum_loss += batch*(loss.item())
    return sum_loss/total

In [35]:
def predict(model, test_dl):
    model.eval()
    x, y = next(iter(test_dl))
    x = x.cuda().float()
    y = y.cuda().long()
    N = x.shape[0]
    h = model.initHidden(N).cuda()
    for t in range(x.shape[1]):
        out, h = model(x[:,t], h)
    loss = F.cross_entropy(out, y)
    _, pred = torch.max(out, 1)
    acc = pred.eq(y).sum().float()/N
    print("test loss %.3f and accuracy %.3f" % (loss.item(), acc.item()))

In [36]:
vocab_size = 55
hidden_size = 80
n_classes = 18
model = CharRNN(vocab_size, hidden_size, n_classes).cuda()

In [37]:
def train_loop(model, lr, epochs=20):
    optim = get_optimizer(model, lr =lr, wd = 0.0)
    for i in range(epochs):
        loss = train(model, optim, train_dl)
        if i%5 == 1: print("train loss %.3f" % loss)
    predict(model, test_dl)

In [38]:
train_loop(model, lr=0.01, epochs=20)

train loss 2.165
train loss 1.891
train loss 1.633
train loss 1.576
test loss 1.348 and accuracy 0.567


In [39]:
train_loop(model, lr=0.001, epochs=20)

train loss 1.354
train loss 1.318
train loss 1.297
train loss 1.272
test loss 1.238 and accuracy 0.611


In [40]:
train_loop(model, lr=0.001, epochs=20)

train loss 1.242
train loss 1.216
train loss 1.197
train loss 1.176
test loss 1.143 and accuracy 0.648


In [41]:
train_loop(model, lr=0.001, epochs=40)

train loss 1.149
train loss 1.132
train loss 1.119
train loss 1.103
train loss 1.088
train loss 1.072
train loss 1.058
train loss 1.043
test loss 1.014 and accuracy 0.691


In [42]:
train_loop(model, lr=0.001, epochs=40)

train loss 1.064
train loss 1.018
train loss 1.005
train loss 0.995
train loss 0.984
train loss 0.973
train loss 0.963
train loss 0.954
test loss 0.928 and accuracy 0.724


In [43]:
train_loop(model, lr=0.001, epochs=40)

train loss 0.952
train loss 0.935
train loss 0.927
train loss 0.920
train loss 0.912
train loss 0.904
train loss 0.897
train loss 0.889
test loss 0.866 and accuracy 0.742


In [ ]:
train_loop(model, lr=0.001, epochs=40)

train loss 0.882
train loss 0.873
train loss 0.870
train loss 0.865
train loss 0.859
train loss 0.853
train loss 0.847
train loss 0.841


# Model with character embeddings 

In [ ]:
class NameDatasetEmb(Dataset):
    def __init__(self, path, vocab2id, label2id, seq_len=15, vocab_len=55):
        self.df = pd.read_csv(path, header=None)
        self.label2id = label2id
        self.vocab2id = vocab2id
        self.seq_len = seq_len
        self.vocab_len = vocab_len 
        self.x = df[0].values
        self.y = [self.label2id[l] for l in df[1].values]
        self.vocab2id = vocab2id
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        x = pad_seq(self.x[idx], self.seq_len, self.vocab2id)
        return x, self.y[idx]

In [ ]:
train_2 = NameDatasetEmb(PATH/"names_train.csv", vocab2id, label2id)
test_2 = NameDatasetEmb(PATH/"names_test.csv", vocab2id, label2id)

In [ ]:
batch_size = 2000
n = len(test)
train_2_dl = DataLoader(train_2, batch_size=batch_size)
test_2_dl = DataLoader(test_2, batch_size=n)

In [ ]:
train_2[0]

In [ ]:
class CharEmbRNN(nn.Module):
    def __init__(self, vocab_size, emb_size, hidden_size, output_size):
        super(CharEmbRNN, self).__init__()
        self.emb = nn.Embedding(vocab_size, emb_size)
        self.hidden_size = hidden_size
        self.linear_i2h = nn.Linear(emb_size + hidden_size, hidden_size)
        self.linear_h2o = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden):
        x = x.long()
        x = self.emb(x)
        combined = torch.cat((x, hidden), 1)
        hidden = F.tanh(self.linear_i2h(combined))
        output = self.linear_h2o(hidden)
        return output, hidden

    def initHidden(self, bash_size):
        return torch.zeros(bash_size, self.hidden_size)

## Train 

In [42]:
def train_loop(model, epochs, lr=0.01, wd=0.0):
    optim = get_optimizer(model, lr = lr, wd = wd)
    for i in range(epochs):
        loss = train(model, optim, train_2_dl)
        if i%5 == 1: print("train loss %.3f" % loss)
    predict(model, test_2_dl)

In [43]:
vocab_size = 55
emb_size = 30
hidden_size = 80
n_classes = 18
model = CharEmbRNN(vocab_size, emb_size, hidden_size, n_classes).cuda()
optim = get_optimizer(model, lr =0.01, wd = 0.0)

In [44]:
train_loop(model, epochs=40, lr=0.01, wd=0.0)

train loss 5.018
train loss 1.601
train loss 1.621
train loss 1.369
train loss 1.287
train loss 1.216
train loss 1.154
train loss 1.099
test loss 0.991 and accuracy 0.692


In [45]:
train_loop(model, epochs=20, lr=0.01, wd=0.0)

train loss 1.255
train loss 1.047
train loss 0.959
train loss 0.897
test loss 0.802 and accuracy 0.762


In [46]:
train_loop(model, epochs=20, lr=0.001, wd=0.0)

train loss 0.805
train loss 0.793
train loss 0.787
train loss 0.779
test loss 0.767 and accuracy 0.771


In [47]:
train_loop(model, epochs=40, lr=0.001, wd=0.00)

train loss 0.773
train loss 0.764
train loss 0.759
train loss 0.753
train loss 0.746
train loss 0.740
train loss 0.733
train loss 0.727
test loss 0.715 and accuracy 0.786


In [58]:
train_loop(model, epochs=20, lr=0.001, wd=0.0)
train_loop(model, epochs=20, lr=0.001, wd=0.0)

train loss 0.379
train loss 0.372
train loss 0.367
train loss 0.362
test loss 0.354 and accuracy 0.891
train loss 0.359
train loss 0.352
train loss 0.348
train loss 0.343
test loss 0.335 and accuracy 0.898


In [59]:
train_loop(model, epochs=20, lr=0.001, wd=0.0)
train_loop(model, epochs=20, lr=0.001, wd=0.0)

train loss 0.340
train loss 0.333
train loss 0.329
train loss 0.325
test loss 0.317 and accuracy 0.904
train loss 0.322
train loss 0.316
train loss 0.312
train loss 0.307
test loss 0.300 and accuracy 0.911


In [60]:
train_loop(model, epochs=20, lr=0.001, wd=0.0)
train_loop(model, epochs=20, lr=0.001, wd=0.0)

train loss 0.306
train loss 0.299
train loss 0.295
train loss 0.291
test loss 0.284 and accuracy 0.917
train loss 0.291
train loss 0.283
train loss 0.280
train loss 0.276
test loss 0.269 and accuracy 0.922


In [61]:
train_loop(model, epochs=20, lr=0.001, wd=0.0)
train_loop(model, epochs=20, lr=0.001, wd=0.0)

train loss 0.276
train loss 0.269
train loss 0.265
train loss 0.261
test loss 0.255 and accuracy 0.927
train loss 0.263
train loss 0.255
train loss 0.251
train loss 0.248
test loss 0.242 and accuracy 0.931


## Exercise
Change the first model to learn a character language model that generates last names.

# References
This notebook is a modified version of this tutorial
http://pytorch.org/tutorials/intermediate/char_rnn_classification_tutorial.html. Here I implement vanilla RNNs.